In [406]:
"""
    Dynamic Edge-Conditioned Filters in Convolutional Neural Networks on Graphs
    https://github.com/mys007/ecc
    https://arxiv.org/abs/1704.02901
    2017 Martin Simonovsky
"""
from __future__ import division
from __future__ import print_function
from builtins import range

import igraph
import torch
from collections import defaultdict
import numpy as np
import torch.nn.init as init

    
class GraphConvInfo(object):          
    """ Holds information about the structure of graph(s) in a vectorized form useful to `GraphConvModule`. 
    
    We assume that the node feature tensor (given to `GraphConvModule` as input) is ordered by igraph vertex id, e.g. the fifth row corresponds to vertex with id=4. Batch processing is realized by concatenating all graphs into a large graph of disconnected components (and all node feature tensors into a large tensor).

    The class requires problem-specific `edge_feat_func` function, which receives dict of edge attributes and returns Tensor of edge features and LongTensor of inverse indices if edge compaction was performed (less unique edge features than edges so some may be reused).
    """

    def __init__(self, *args, **kwargs):
        self._idxn = None           #indices into input tensor of convolution (node features)
        self._idxe = None           #indices into edge features tensor (or None if it would be linear, i.e. no compaction)
        self._degrees = None        #in-degrees of output nodes (slices _idxn and _idxe)
        self._degrees_gpu = None
        self._edgefeats = None      #edge features tensor (to be processed by feature-generating network)
        if len(args)>0 or len(kwargs)>0:
            self.set_batch(*args, **kwargs)
    def set_batch(self, graphs, edge_feat_func):
        """ Creates a representation of a given batch of graphs.
        
        Parameters:
        graphs: single graph or a list/tuple of graphs.
        edge_feat_func: see class description.
        """
        
        graphs = graphs if isinstance(graphs,(list,tuple)) else [graphs]
        p = 0
        idxn = []
        degrees = []
        edgeattrs = defaultdict(list)
                
        for i,G in enumerate(graphs):
            #E = np.array(G.get_edgelist())
            E = np.array(list(G.edges()))
            idx = E[:,1].argsort() # sort by target
            
            idxn.append(p + E[idx,0])
            
            # edgeseq = G.es[idx.tolist()]
            # for a in G.es.attributes():
            #     edgeattrs[a] += edgeseq.get_attribute_values(a)
            
            edgelist = list(graph.edges(data=False))  # Get the list of edges
            edgeseq = [graph.get_edge_data(u, v) for u, v in edgelist]  # Get the sequence of edge data
            # degrees += G.indegree(G.vs, loops=True)
            # p += G.vcount()
            degrees += list(dict(G.in_degree()).values())  # Get the in-degrees of the nodes
            p += G.number_of_nodes()  # Accumulate the node count

              
        self._edgefeats, self._idxe = edge_feat_func(edgelist)
        
        self._idxn = torch.LongTensor(np.concatenate(idxn))
        if self._idxe is not None:
            assert self._idxe.numel() == self._idxn.numel()
            
        self._degrees = torch.LongTensor(degrees)
        self._degrees_gpu = None            
        
    def cuda(self):
        self._idxn = self._idxn.cuda()
        if self._idxe is not None: self._idxe = self._idxe.cuda()
        self._degrees_gpu = self._degrees.cuda()
        self._edgefeats = self._edgefeats.cuda()        
        
    def get_buffers(self):
        """ Provides data to `GraphConvModule`.
        """
        return self._idxn, self._idxe, self._degrees, self._degrees_gpu, self._edgefeats

    
    
    
    
"""
    Dynamic Edge-Conditioned Filters in Convolutional Neural Networks on Graphs
    https://github.com/mys007/ecc
    https://arxiv.org/abs/1704.02901
    2017 Martin Simonovsky
"""
from __future__ import division
from __future__ import print_function
from builtins import range

import torch
import torch.nn as nn
from torch.autograd import Variable, Function
import cuda_kernels




class GraphConvFunction(Function):
    """ Computes operations for each edge and averages the results over respective nodes. The operation is either matrix-vector multiplication (for 3D weight tensors) or element-wise vector-vector multiplication (for 2D weight tensors). The evaluation is computed in blocks of size `edge_mem_limit` to reduce peak memory load. See `GraphConvInfo` for info on `idxn, idxe, degs`.
    """
    def __init__(self, in_channels, out_channels, idxn, idxe, degs, degs_gpu, edge_mem_limit=1e20):
        super(Function, self).__init__()
        self._in_channels = in_channels
        self._out_channels = out_channels
        self._idxn = idxn
        self._idxe = idxe
        self._degs = degs
        self._degs_gpu = degs_gpu
        self._shards = get_edge_shards(degs, edge_mem_limit)

    def _multiply(self, a, b, out, f_a=None, f_b=None):
        """ Performs operation on edge weights and node signal """
        if b.dim() == 3:
            # weights are full in_channels x out_channels matrices -> mm
            torch.bmm(f_a(a) if f_a else a, f_b(b) if f_b else b, out=out)
        else:
            # weights represent diagonal matrices -> mul
            torch.mul(a, b.expand_as(a), out=out)
    @staticmethod
    def forward(ctx, input, weights, in_channels, out_channels, idxn, idxe, degs, degs_gpu, edge_mem_limit):
        ctx.save_for_backward(input, weights)
        ctx.in_channels = in_channels
        ctx.out_channels = out_channels
        ctx.idxn = idxn
        ctx.idxe = idxe
        ctx.degs = degs
        ctx.degs_gpu = degs_gpu
        ctx.edge_mem_limit = edge_mem_limit

        full_weight_mat = weights.dim() == 3
        assert full_weight_mat or (in_channels == out_channels and weights.size(1) == in_channels)

        output = input.new(degs.numel(), out_channels)

        # loop over blocks of output nodes
        startd, starte = 0, 0
        for numd, nume in get_edge_shards(degs, edge_mem_limit):

            # select sequence of matching pairs of node and edge weights
            sel_input = torch.index_select(input, 0, idxn.narrow(0, starte, nume))

            if idxe is not None:
                sel_weights = torch.index_select(weights, 0, idxe.narrow(0, starte, nume))
            else:
                sel_weights = weights.narrow(0, starte, nume)

            # compute matrix-vector products
            products = input.new()
            GraphConvFunction._multiply(sel_input, sel_weights, products, lambda a: a.unsqueeze(1))

            # average over nodes
            if idxn.is_cuda:
                cuda_kernels.conv_aggregate_fw(output.narrow(0, startd, numd), products.view(-1, out_channels),
                                                degs_gpu.narrow(0, startd, numd))
            else:
                k = 0
                for i in range(startd, startd + numd):
                    if degs[i] > 0:
                        torch.mean(products.narrow(0, k, degs[i]), 0, out=output[i])
                    else:
                        output[i].fill_(0)
                    k = k + degs[i]

            startd += numd
            starte += nume
            del sel_input, sel_weights, products

        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, weights = ctx.saved_tensors

        grad_input = input.new(input.size()).fill_(0)
        grad_weights = weights.new(weights.size())
        if ctx.idxe is not None:
            grad_weights.fill_(0)

        # loop over blocks of output nodes
        startd, starte = 0, 0
        for numd, nume in get_edge_shards(ctx.degs, ctx.edge_mem_limit):

            grad_products, tmp = input.new(nume, ctx.out_channels), input.new()

            if ctx.idxn.is_cuda:
                cuda_kernels.conv_aggregate_bw(grad_products, grad_output.narrow(0, startd, numd),
                                                ctx.degs_gpu.narrow(0, startd, numd))
            else:
                k = 0
                for i in range(startd, startd + numd):
                    if ctx.degs[i] > 0:
                        torch.div(grad_output[i], ctx.degs[i], out=grad_products[k])
                        if ctx.degs[i] > 1:
                            grad_products.narrow(0, k + 1, ctx.degs[i] - 1).copy_(
                                grad_products[k].expand(ctx.degs[i] - 1, 1, ctx.out_channels).squeeze(1))
                        k = k + ctx.degs[i]

            # grad wrt weights
            sel_input = torch.index_select(input, 0, ctx.idxn.narrow(0, starte, nume))

            if ctx.idxe is not None:
                GraphConvFunction._multiply(sel_input, grad_products, tmp, lambda a: a.unsqueeze(1).transpose_(2, 1),
                                            lambda b: b.unsqueeze(1))
                grad_weights.index_add_(0, ctx.idxe.narrow(0, starte, nume), tmp)
            else:
                GraphConvFunction._multiply(sel_input, grad_products, grad_weights.narrow(0, starte, nume),
                                            lambda a: a.unsqueeze(1).transpose_(2, 1), lambda b: b.unsqueeze(1))

            # grad wrt input
            if ctx.idxe is not None:
                torch.index_select(weights, 0, ctx.idxe.narrow(0, starte, nume), out=tmp)
                GraphConvFunction._multiply(grad_products, tmp, sel_input, lambda a: a.unsqueeze(1),
                                            lambda b: b.transpose(2, 1))
                del tmp
            else:
                GraphConvFunction._multiply(grad_products, weights.narrow(0, starte, nume), sel_input,
                                            lambda a: a.unsqueeze(1), lambda b: b.transpose(2, 1))

        return grad_input, grad_weights, None, None, None, None, None, None, None

In [506]:
class GraphConvModule(nn.Module):
    """ Computes graph convolution using filter weights obtained from a filter generating network (`filter_net`).
        The input should be a 2D tensor of size (# nodes, `in_channels`). Multiple graphs can be concatenated in the same tensor (minibatch).
    
    Parameters:
    in_channels: number of input channels
    out_channels: number of output channels
    filter_net: filter-generating network transforming a 2D tensor (# edges, # edge features) to (# edges, in_channels*out_channels) or (# edges, in_channels)
    gc_info: GraphConvInfo object containing graph(s) structure information, can be also set with `set_info()` method.
    edge_mem_limit: block size (number of evaluated edges in parallel) for convolution evaluation, a low value reduces peak memory. 
    """

    def __init__(self, in_channels, out_channels, filter_net, gc_info=None, edge_mem_limit=1e20):
        super(GraphConvModule, self).__init__()
        
        self._in_channels = in_channels
        self._out_channels = out_channels
        self._fnet = filter_net
        self._edge_mem_limit = edge_mem_limit
        
        self.set_info(gc_info)
        
    def set_info(self, gc_info):
        self._gci = gc_info
    
    def forward(self, input):       
        # get graph structure information tensors
        idxn, idxe, degs, degs_gpu, edgefeats = self._gci.get_buffers()
        edgefeats = Variable(edgefeats, requires_grad=False)
        
        # evalute and reshape filter weights
        weights = self._fnet(edgefeats)
        print(input.dim())
        print(weights.shape)
        assert input.dim()==2 and weights.dim()==2 and (weights.size(1) == self._in_channels*self._out_channels or
               (self._in_channels == self._out_channels and weights.size(1) == self._in_channels))
        if weights.size(1) == self._in_channels*self._out_channels:
            weights = weights.view(-1, self._in_channels, self._out_channels)
        return GraphConvFunction.apply(input, weights, self._in_channels, self._out_channels, idxn, idxe, degs, degs_gpu, self._edge_mem_limit)


In [507]:
#MLP
def create_fnet(widths, nfeat, nfeato, orthoinit, llbias):
    """ Creates feature-generating network, a multi-layer perceptron.
    Parameters:
    widths: list of widths of hidden layers
    nfeat, nfeato: # input and output channels of the convolution
    orthoinit: whether to use orthogonal weight initialization
    llbias: whether to use bias in the last layer
    """
    fnet_modules = []   
    for k in range(len(widths)-1):
        fnet_modules.append(nn.Linear(widths[k], widths[k+1]))
        if orthoinit: init.orthogonal(fnet_modules[-1].weight, gain=init.calculate_gain('relu'))
        fnet_modules.append(nn.ReLU(True))                    
    fnet_modules.append(nn.Linear(widths[-1], nfeat*nfeato, bias=llbias))
    if orthoinit: init.orthogonal(fnet_modules[-1].weight)
    return nn.Sequential(*fnet_modules)
            

In [508]:
def get_edge_shards(degs, edge_mem_limit):
    """ Splits iteration over nodes into shards, approximately limited by `edge_mem_limit` edges per shard. 
    Returns a list of pairs indicating how many output nodes and edges to process in each shard."""
    d = degs if isinstance(degs, np.ndarray) else degs.numpy()
    cs = np.cumsum(d)
    cse = cs // edge_mem_limit
    _, cse_i, cse_c = np.unique(cse, return_index=True, return_counts=True)
    
    shards = []
    for b in range(len(cse_i)):
        numd = cse_c[b]
        nume = (cs[-1] if b==len(cse_i)-1 else cs[cse_i[b+1]-1]) - cs[cse_i[b]] + d[cse_i[b]]   
        shards.append( (int(numd), int(nume)) )
    return shards

In [509]:
def adjacency_matrix_to_edges(adjacency_matrix):
    edges = []
    num_nodes = adjacency_matrix.shape[0]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if adjacency_matrix[i, j] != 0:
                edges.append((i, j))
    return edges

In [510]:
adj=np.load('/volume1/home/rzhu/gcn/data/volume/0308/000046_XSHE_3_3_graph_input.npy')
edges=adjacency_matrix_to_edges(adj_transposed)


In [511]:
import numpy as np
import networkx as nx
adj_transposed = np.transpose(adj)
# Your adjacency matrix
adjacency_matrix = np.array(adj_transposed)
# Create a directed graph from the adjacency matrix
graph = nx.DiGraph(adjacency_matrix)
print(graph.edges())

[(0, 0), (0, 1), (0, 4), (0, 5), (1, 1), (1, 2), (1, 6), (1, 12), (2, 2), (2, 3), (2, 7), (2, 13), (3, 3), (3, 14), (4, 4), (4, 5), (4, 8), (4, 9), (5, 5), (5, 6), (5, 10), (6, 6), (6, 7), (6, 11), (7, 7), (8, 8), (8, 9), (8, 12), (8, 13), (9, 9), (9, 10), (9, 14), (10, 10), (10, 11), (10, 15), (11, 11), (12, 12), (12, 13), (13, 13), (13, 14), (14, 14), (14, 15)]


In [512]:
import torch
def edge_feat_func(edges):
    edge_dict = {}  # 用于存储边的差异及其对应的索引
    idxe = []  # 用于存储边特征的索引
    edgefeats = []  # 用于存储边特征的 one-hot 编码

    # 遍历边列表，计算边的差异并将其分类
    for edge in edges:
        diff = edge[1] - edge[0]  # 计算边的差异
        if diff not in edge_dict:
            edge_dict[diff] = len(edge_dict)  # 如果差异之前没有出现过，则添加到字典中
        idxe.append(edge_dict[diff])  # 将差异的索引添加到 idxe 中

    # 构建 one-hot 编码的边特征
    num_features = len(edge_dict)
    seen_feats = set()  # 用于记录已经出现过的特征索引
    for idx in idxe:
        if idx not in seen_feats:  # 检查特征索引是否已经存在于集合中
            feat = torch.zeros(num_features)
            feat[idx] = 1
            edgefeats.append(feat)
            seen_feats.add(idx)  # 将特征索引添加到集合中

    return torch.stack(edgefeats), torch.LongTensor(idxe)

In [513]:
edgefeats= edge_feat_func(edges)

In [514]:
edgefeats, idxe = edge_feat_func(edges) 
print("Edge Features:")
print(edgefeats)
print("Index List:")
print(idxe)

Edge Features:
tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
Index List:
tensor([0, 1, 2, 3, 0, 1, 3, 4, 0, 1, 3, 4, 0, 4, 0, 1, 2, 3, 0, 1, 3, 0, 1, 3,
        0, 0, 1, 2, 3, 0, 1, 3, 0, 1, 3, 0, 0, 1, 0, 1, 0, 1])


In [515]:
gc_info = GraphConvInfo()
gc_info.set_batch(graph, edge_feat_func)

In [522]:
fnet = create_fnet([5,5], 1, 1, 1, 1)
gconv = GraphConvModule(7, 7, fnet, edge_mem_limit=1e20)
gconv.set_info(gc_info)

/tmp/ipykernel_2051659/3782370251.py:13: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  if orthoinit: init.orthogonal(fnet_modules[-1].weight, gain=init.calculate_gain('relu'))
/tmp/ipykernel_2051659/3782370251.py:16: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  if orthoinit: init.orthogonal(fnet_modules[-1].weight)


In [524]:
inputs_data=np.load('/volume1/home/rzhu/gcn/data/volume/0308/000046_XSHE_3_3_inputs.npy', allow_pickle=True)
inputs_data = [[[torch.tensor(x, dtype=torch.float64) for x in sublist] for sublist in list1] for list1 in inputs_data]
array_data = np.array(inputs_data)
inputs = np.reshape(array_data, (len(inputs_data), 16,-1))

In [525]:
inputs[0].shape

(16, 7)

In [526]:
inputs_tensor = torch.tensor(inputs[0], dtype=torch.float32)
inputs_tensor.shape

torch.Size([16, 7])

In [527]:
output = gconv(inputs_tensor)

2
torch.Size([5, 1])


AssertionError: 